In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit, minimize
from scipy.interpolate import splrep, BSpline
from pgmpy.inference import BeliefPropagation

import src.modelling_fev1.pred_fev1 as pred_fev1
import src.data.breathe_data as breathe_data
import src.modelling_fev1.effort_corrected_fev1 as effort_corrected_fev1
import src.models.builders as lung_health_models

plotsdir = "../../../../PlotsBreathe/O2_modelling/"

In [2]:
df = breathe_data.build_O2_FEV1_df()


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  233
Number of rows:  26812
Number of FEV1 recordings: 23778
Number of O2 Saturation recordings: 23431
Dropped 0 entries with NaN O2 Saturation and NaN FEV1
Dropped 3381 entries with NaN O2 Saturation
Dropped 3034 entries with NaN FEV1
20397 entries remain


/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ecFEV1"][mask] = smooth.smooth_vector(df.FEV1[mask].to_numpy(), "max").copy()
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ecFEV1"][mask] = smooth.smooth_vector(df.FEV1[mask].to_numpy(), "max").copy()
/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/O2_modelling/../data/effort_corrected_fev1.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 149.50534972488947
Built data structure with 213 IDs and 20397 entries


In [4]:
# Save to excel
df.to_excel(plotsdir + "Breathe_O2_FEV1.xlsx", index=False)

In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125


In [4]:
df.describe()

,FEV1,O2 Saturation,ecFEV1,Age,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
count,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000,20397.000000
mean,2.198363,96.966501,2.263423,34.801147,166.247120,3.507772,97.711339,64.659185,62.787907,99.238755
std,0.816148,1.649808,0.822063,10.154773,9.151066,0.649323,0.449598,20.301483,20.271387,1.686950
min,0.490000,76.000000,0.500000,18.000000,143.000000,2.213968,96.975001,15.320382,15.013975,77.519654
25%,1.550000,96.000000,1.610000,27.000000,160.000000,2.979444,97.225960,48.198629,46.536607,98.699543
50%,2.030000,97.000000,2.090000,34.000000,166.000000,3.386997,97.989462,62.880920,61.235539,99.767593
75%,2.760000,98.000000,2.830000,41.000000,173.000000,3.987357,98.114941,77.657779,75.719884,100.255981
max,5.260000,100.000000,5.260000,64.000000,193.000000,5.322753,98.340804,149.505350,149.505350,103.026043


## Factor - Airway resistance vs O2 drop

In [365]:
# Infer airway resistance using the model
def infer_airway_resistance_for_ID(df_for_ID):
    print(
        f"\nRunning for ID {df_for_ID.ID.iloc[0]}, with {len(df_for_ID)} observations"
    )
    airway_resistances_for_ID = np.array([])
    # TODO: update predFEV1 sigma!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    healthy_FEV1_prior = {
        "type": "gaussian",
        "mu": df_for_ID["Predicted FEV1"].iloc[0],
        "sigma": 0.4,
    }
    healthy_o2sat_prior = {
        "type": "gaussian",
        "mu": 0.98,
        "sigma": 0.01,
    }
    (
        model,
        HFEV1,
        _,
        ecFEV1,
        _,
        _,
        _,
        AR,
        prior_ar,
    ) = lung_health_models.build_FEV1_O2_point_in_time_model(
        healthy_FEV1_prior, healthy_o2sat_prior
    )
    inf_alg = BeliefPropagation(model)
    for ecFEV1_obs in df_for_ID.ecFEV1:
        res_ar = lung_health_models.infer(
            inf_alg, [AR], [[ecFEV1, ecFEV1_obs]], show_progress=False
        )
        # Get argmax of res_ar
        idx = np.argmax(res_ar.values)
        most_probable_airway_resistance = round(AR.bins[idx], 2)
        most_probable_airway_resistance_bin_str = AR.bins_str[idx]
        # print(f"Most probable airway resistance: {most_probable_airway_resistance}, bin: {most_probable_airway_resistance_bin_str}")
        # Add to airway resistance array
        airway_resistances_for_ID = np.append(
            airway_resistances_for_ID, most_probable_airway_resistance
        )
    return airway_resistances_for_ID


s_AW = df.groupby(["ID"]).apply(infer_airway_resistance_for_ID)


Running for ID 101, with 816 observations
*** Building lung model with HFEV1 and AB ***
Defining gaussian prior with mu = 3.61, sigma = 0.4


KeyboardInterrupt: 

In [5]:
# Read s_AW from excel and add to df
# s_AW.to_excel(f"{plotsdir}airway_resistance.xlsx")
s_AW = pd.read_excel(f"{plotsdir}airway_resistance.xlsx", index_col=0)[0]
s_AW = s_AW.apply(
    lambda x: [
        float(i)
        for i in x.replace("\n", "").replace("  ", " ").strip("[]").split(" ")
        if i != ""
    ]
)
df["Airway Resistance (%)"] = pd.Series(sum(s_AW.to_list(), [])) * 100

In [8]:
# Check that the computed airway resistance makes sense
# Plot ID 101 FEV1 profile with Date Recorded
# '113', '126', '202', '331'
ID = "101"
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["FEV1"],
        mode="markers",
        name="FEV1",
    )
)
# Add trace for predicted FEV1
# fig.add_trace(go.Scatter(x=df[df.ID == "101"]["Date Recorded"], y=df[df.ID == "101"]["Predicted FEV1"], mode="markers", name="Predicted FEV1"))
# Add trace for airway resistance using s_AW
fig.add_trace(
    go.Scatter(
        x=df[df.ID == ID]["Date Recorded"],
        y=df[df.ID == ID]["Airway Resistance (%)"] / 100,
        mode="markers",
        name="Airway Resistance (%)",
    )
)
# fig.add_trace(go.Scatter(x=df[df.ID == ID]["Date Recorded"], y=s_AW[ID], mode="markers", name="Airway Resistance"))

In [153]:
# # 0% airway resistance: FEV1 = Predicted FEV1
# # 25% airway resistance: FEV1 = 0.75 * Predicted FEV1
# # Negative airway resistance: FEV1 > Predicted FEV1
# # Airway resistance = 1 - FEV1/Predicted FEV1
df["Airway Resistance Computed (%)"] = 100 - df["ecFEV1 % Predicted"]

df["Drop from Healthy O2 Saturation (%)"] = (
    df["O2 Saturation"] - df["Healthy O2 Saturation"]
)
df["O2 Saturation % Predicted"] = (
    df["O2 Saturation"] / df["Healthy O2 Saturation"] * 100
)

df.head()

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%),Sex/height corrected O2 Saturation (%),O2 Saturation % Predicted,O2 Saturation %
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,63.0,63.435523,-0.22596,99.767593,99.767593,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,66.0,63.435523,-1.22596,98.739061,98.739061,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,63.0,63.435523,-1.22596,98.739061,98.739061,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,66.0,63.158519,-0.22596,99.767593,99.767593,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,63.0,62.327508,0.77404,100.796125,100.796125,100.796125


### Raw scatter plot

In [1]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Computed Airway Resistance vs  ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(
    df,
    x="Airway Resistance Computed (%)",
    y="O2 Saturation % Predicted",
    # y="Drop from Healthy O2 Saturation (%)",
    title=title,
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

NameError: name 'df' is not defined

In [189]:
# Plot Airway resistance vs O2 drop
title = f"O2 Drop vs Inferred Airway Resistance ({df.ID.nunique()} IDs, {len(df)} datapoints)"
fig = px.scatter(
    df,
    x="Airway Resistance (%)",
    # y="Drop from Healthy O2 Saturation (%)",
    y="O2 Saturation % Predicted",
    title=title,
)
# Reduce marker size
fig.update_traces(marker=dict(size=2))
fig.update_layout(font=dict(size=10))
fig.show()
fig.write_image(f"{plotsdir}{title}.pdf")
# 100% is about 98 for females and 97.4 for males
# Hence, threshold at 95% => 3-3.5% drop in O2 Saturation => 96.5-97% O2 Saturation

### Compute and interpolate factor profile

In [206]:
# Group by Airway Resistance and take 80th percentile of O2 Sat / Healthy O2 Sat if there are more than 50 observations
def calc_rmax_o2(df_for_AR, rmax_o2_sat_col, percentile=80):
    return np.percentile(df_for_AR[rmax_o2_sat_col], percentile), len(df_for_AR)
    # Take data between 80 and 90th percentile
    # return np.percentile(
    #     df_for_AR[rmax_o2_sat_col],
    #     range(percentile - 5, percentile + 5),
    # ).mean(), len(df_for_AR)


def fit_factor_profile(df_to_fit, rmax_o2_sat_col):
    x_data = df_to_fit["Airway Resistance (%)"].values
    y_data = df_to_fit[rmax_o2_sat_col].values

    # Piecewise fit (constant + polynomial)
    def func(x, x0, y0, k1, k2, k3):
        # x0 = 43
        # y0 = df_to_fit[df_to_fit["Airway Resistance (%)"] < x0][
        #     o2_col
        # ].mean()

        return np.piecewise(
            x,
            [x <= x0],
            [
                lambda x: y0,
                lambda x: k1 * np.power((x - x0), 3)
                + k2 * np.power((x - x0), 2)
                + k3 * (x - x0)
                + y0,
            ],
        )

    # def objective(params, x, y):
    #     return np.sum((func(x, *params) - y)**2)

    # Enforce monotonicity constraint
    # constraints = ({'type': 'ineq', 'fun': lambda params: np.diff(func(x_data, *params))})

    # Initial guess for parameters
    # initial_guess = [4.34232599e+01, 8.92599726e-01, -3.60069643e-04, 1.56798589e-02, -2.12605357e-01]

    # # Minimize the objective function with the constraint
    # result = minimize(objective, initial_guess, args=(x_data, y_data), constraints=constraints)
    # parameters = result.x

    parameters, covariance = curve_fit(
        func,
        df_to_fit["Airway Resistance (%)"].values,
        df_to_fit[rmax_o2_sat_col].values,
    )
    print(f"Parameters: {parameters}")
    df_to_fit["Piecewise fit"] = func(x_data, *parameters)

    # Spline fit
    ## Base value for smoothing parameter
    s = df_to_fit.shape[0] - np.sqrt(2 * df_to_fit.shape[0])
    print(f"Smoothing parameter: {s}")
    ### Create a spline representation of the curve
    ### tck-tuple: (t,c,k) containing the vector of knots, the B-spline coefficients, and the degree of the spline.
    tck = splrep(
        x_data,
        y_data,
        s=0,
    )
    ### Evalute the spline repr on a new set of points
    df_to_fit["Spline"] = BSpline(*tck)(df_to_fit["Airway Resistance (%)"])

    # Mean smoothing
    df_to_fit["Mean Smoothing"] = (
        df_to_fit[rmax_o2_sat_col].rolling(5, center=True).mean()
    )
    return df_to_fit


df_for_AW_O2Sat_study = df[df.ID != "120"].copy()

for prctile in [10, 85, 90, 95, 98, 100]:  # range(60, 90, 5):
    rmax_o2_sat_col = f"{prctile}th-rmax O2 Saturation<br> % Predicted"

    rmax_AW_O2Sat = df_for_AW_O2Sat_study.groupby(["Airway Resistance (%)"]).apply(
        lambda x: calc_rmax_o2(x, "O2 Saturation % Predicted", prctile)
    )
    # Unstack rmax_AR_O2Sat tuples into 2 columns
    rmax_AW_O2Sat = (
        rmax_AW_O2Sat.apply(pd.Series)
        .rename(columns={0: rmax_o2_sat_col, 1: "#datapoints"})
        .reset_index()
    )
    # Add column for >50 datapoints
    rmax_AW_O2Sat[">50 datapoints"] = rmax_AW_O2Sat["#datapoints"] > 50
    # Mask for >50 datapoints
    rmax_AW_O2Sat_plot = fit_factor_profile(
        rmax_AW_O2Sat[rmax_AW_O2Sat[">50 datapoints"]].copy(), rmax_o2_sat_col
    )

    # PLot rmax_AW_O2Sat
    title = f"Max achievable O2 Saturation % Predicted ({prctile}th-rmax) vs Airway Resistance ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=rmax_AW_O2Sat_plot["Airway Resistance (%)"],
            y=rmax_AW_O2Sat_plot[rmax_o2_sat_col],
            mode="markers",
            name="Airway Resistance vs O2 drop",
        ),
    )
    # fig.add_trace(
    #     go.Scatter(
    #         x=rmax_AW_O2Sat_plot["Airway Resistance (%)"],
    #         y=rmax_AW_O2Sat_plot["Spline"],
    #         mode="lines",
    #         name="Spline",
    #     )
    # )
    # fig.add_trace(
    #     go.Scatter(
    #         x=rmax_AW_O2Sat_plot["Airway Resistance (%)"],
    #         y=rmax_AW_O2Sat_plot["Mean Smoothing"],
    #         mode="lines",
    #         name="Mean Smoothing",
    #     )
    # )
    fig.add_trace(
        go.Scatter(
            x=rmax_AW_O2Sat_plot["Airway Resistance (%)"],
            y=rmax_AW_O2Sat_plot["Piecewise fit"],
            mode="lines",
            name="Constant + 3rd order polynomial fit",
        ),
    )
    fig.update_traces(line=dict(width=1), marker=dict(size=3))
    fig.update_yaxes(
        range=[90, 103.5],
        nticks=10,
        title=rmax_o2_sat_col,
    )
    fig.update_xaxes(title="Airway Resistance (%)", range=[-5, 91], nticks=10)
    fig.update_layout(title=title, height=300, width=800, font=dict(size=8))
    fig.show()
    # Save to file
    fig.write_image(
        f"{plotsdir}{title}.pdf",
        width=1000,
        height=400,
    )

# Bar plot of number of datapoints per airway resistance bin
title = f"Number of datapoints per Airway Resistance bin ({df_for_AW_O2Sat_study.ID.nunique()} IDs, {len(df_for_AW_O2Sat_study)} datapoints)"
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=rmax_AW_O2Sat["Airway Resistance (%)"],
        y=rmax_AW_O2Sat["#datapoints"],
        name="#datapoints",
        marker=dict(color="grey"),
    ),
)
# Add line for 50 datapoints
fig.add_trace(
    go.Scatter(
        x=rmax_AW_O2Sat["Airway Resistance (%)"],
        y=np.repeat(50, len(rmax_AW_O2Sat)),
        mode="lines",
        name="50 datapoints",
    )
)
# Log y axis
fig.update_yaxes(type="log", title="#datapoints", nticks=3)
# Range x axis to 91
fig.update_xaxes(range=[-5, 91], nticks=10, title="Binned Airway Resistance (%)")
fig.update_layout(font=dict(size=8), height=250, width=800, title=title)
fig.show()

Parameters: [ 3.00009169e+00  9.91391086e+01 -1.14178223e-05  4.12204099e-04
 -3.92646033e-02]
Smoothing parameter: 62.75255128608411


Parameters: [ 4.50539786e+01  1.00909074e+02 -4.47552758e-04  2.02209060e-02
 -2.86057990e-01]
Smoothing parameter: 62.75255128608411


Parameters: [ 4.85421719e+01  1.00987280e+02 -6.24322394e-04  2.52952884e-02
 -3.13385623e-01]
Smoothing parameter: 62.75255128608411


Parameters: [ 2.80000037e+01  1.01519426e+02 -9.97388370e-05  6.06868737e-03
 -1.21481443e-01]
Smoothing parameter: 62.75255128608411


Parameters: [ 4.46845638e+01  1.01809234e+02 -4.53567304e-04  2.05847838e-02
 -2.83418263e-01]
Smoothing parameter: 62.75255128608411


Parameters: [-2.31675626e-01  1.02307494e+02 -1.66202975e-05  9.09399128e-04
 -9.86789599e-03]
Smoothing parameter: 62.75255128608411


In [457]:
rmax_AW_O2Sat.sort_values(by="n datapoints").head(10)

,Airway Resistance (%),Drop from Healthy O2 Saturation (%),n datapoints,>50 datapoints
82,86.0,92.538919,3.0,False
80,81.0,100.451352,3.0,False
77,77.0,99.154555,5.0,False
1,1.0,100.698654,6.0,False
79,80.0,97.674663,7.0,False
75,75.0,100.659989,14.0,False
78,78.0,99.741848,15.0,False
74,74.0,98.758706,17.0,False
7,7.0,101.003246,54.0,True
76,76.0,99.779589,54.0,True


### Specific cases for the plot with Airway resistance computed

In [27]:
# Plot FEV1 % Predicted with time for individual 122
def plot_fev1_o2(df, ids, o2_col="O2 Saturation"):
    for id in ids:
        df_for_ID = df[df.ID == id]
        # Create subplot with 2 rows
        fig = make_subplots(rows=2, cols=1)
        # Add trace for FEV1 % Predicted on one subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID["ecFEV1 % Predicted"],
                mode="markers",
                name="ecFEV1 % Predicted",
            ),
            row=1,
            col=1,
        )
        # fig.add_trace(
        #     go.Scatter(
        #         x=df_for_ID["Date Recorded"],
        #         y=df_for_ID["FEV1 % Predicted"],
        #         mode="markers",
        #         name="FEV1 % Predicted",
        #     ),
        #     row=1,
        #     col=1,
        # )
        # Add trace for O2 Saturation on another subplot
        fig.add_trace(
            go.Scatter(
                x=df_for_ID["Date Recorded"],
                y=df_for_ID[o2_col],
                mode="markers",
                name="O2 Saturation",
            ),
            row=2,
            col=1,
        )
        fig.update_traces(marker=dict(size=3), line=dict(width=0.5))
        title = f"ecFEV1 % Predicted and {o2_col} for individual {id} ({len(df_for_ID)} datapoints)"
        fig.update_layout(title=title)
        # Add trace for O2 Saturation on another subplot

        fig.show()

#### Low airway resistance

In [9]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] < -20]
# '113', '126', '202', '331'

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
2782,113,2020-11-23,4.05,98.0,4.92,26,Male,165.5,3.923640,97.320070,125.393781,103.220490,100.698654,-25.393781,100.698654
2783,113,2020-11-24,4.92,98.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,125.393781,100.698654,-32.275147,100.698654
2784,113,2020-12-07,5.19,99.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,132.275147,101.726191,-32.275147,101.726191
2785,113,2020-12-08,5.15,98.0,5.19,26,Male,165.5,3.923640,97.320070,132.275147,131.255685,100.698654,-32.275147,100.698654
2786,113,2020-12-09,4.63,99.0,5.15,26,Male,165.5,3.923640,97.320070,131.255685,118.002684,101.726191,-31.255685,101.726191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13720,202,2021-04-23,2.80,98.0,2.80,64,Female,157.0,2.213968,98.165133,126.469782,126.469782,99.831781,-26.469782,99.831781
13721,202,2021-04-30,1.83,100.0,2.80,64,Female,157.0,2.213968,98.165133,126.469782,82.657036,101.869164,-26.469782,101.869164
20107,331,2021-05-26,2.62,99.0,3.02,54,Female,157.0,2.472585,98.165133,122.139372,105.961972,100.850472,-22.139372,100.850472
20108,331,2021-05-29,3.02,97.0,3.02,54,Female,157.0,2.472585,98.165133,122.139372,122.139372,98.813089,-22.139372,98.813089


In [12]:
plot_fev1_o2(df, ["113", "126", "202", "331"])

#### High airway resistance

In [202]:
# Filter airway resistance below 40%
df[df["Airway Resistance (%)"] > 80].ID.unique()
# 3 individuals '122', '198', '286' have airway resistance > 80%

array(['122', '286'], dtype=object)

In [203]:
# Plot FEV1 % Predicted with time for individual 122
plot_fev1_o2(df, ["122", "286"])

#### High O2 Drop

In [16]:
df[df["Drop from Healthy O2 Saturation (%)"] < 90].ID.unique()

array(['110', '111', '180', '352'], dtype=object)

In [17]:
plot_fev1_o2(df, ["111", "180", "352"])

#### High positive O2 Drop

In [50]:
df[df["Drop from Healthy O2 Saturation (%)"] > 2.8].ID.unique()

array(['185'], dtype=object)

In [37]:
df[df.ID == "120"].sort_values(by="Drop from Healthy O2 Saturation (%)")

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),Airway Resistance Computed (%),Drop from Healthy O2 Saturation (%)
3862,120,2019-06-19,1.68,95.0,1.79,30,Male,182.0,4.728022,97.113028,37.859387,35.532833,97.824156,65.0,62.140613,-2.113028
3922,120,2019-09-25,1.95,95.0,1.95,30,Male,182.0,4.728022,97.113028,41.243467,41.243467,97.824156,59.0,58.756533,-2.113028
3902,120,2019-08-20,1.63,95.0,1.71,30,Male,182.0,4.728022,97.113028,36.167348,34.475308,97.824156,65.0,63.832652,-2.113028
3911,120,2019-09-02,1.75,95.0,1.75,30,Male,182.0,4.728022,97.113028,37.013367,37.013367,97.824156,63.0,62.986633,-2.113028
3899,120,2019-08-08,1.70,95.0,1.70,30,Male,182.0,4.728022,97.113028,35.955843,35.955843,97.824156,63.0,64.044157,-2.113028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110,120,2021-01-29,2.12,100.0,2.17,30,Male,182.0,4.728022,97.113028,45.896576,44.839051,102.972795,55.0,54.103424,2.886972
4112,120,2021-02-04,2.09,100.0,2.17,30,Male,182.0,4.728022,97.113028,45.896576,44.204536,102.972795,57.0,54.103424,2.886972
4116,120,2021-02-18,2.15,100.0,2.15,30,Male,182.0,4.728022,97.113028,45.473566,45.473566,102.972795,55.0,54.526434,2.886972
4131,120,2021-04-16,1.88,100.0,2.06,30,Male,182.0,4.728022,97.113028,43.570021,39.762932,102.972795,61.0,56.429979,2.886972


In [34]:
plot_fev1_o2(df, ["120"], "Drop from Healthy O2 Saturation (%)")
plot_fev1_o2(df, ["120"])

### Factor function

In [20]:
# We wanna plot a noise-agnostic version of the raw scatter plot's top envelope

# > 50 datapoints per bin

# Discretise airway resistance
# Take 90th percentile for each bin
# Plot 90th percentile and amount of data per bin

df_aw_o2_factor

,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,Airway Resistance (%),O2 Sat / Healthy O2 Sat (%)
0,101,2019-02-20,1.31,97.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.287474,99.767593,63.435523,99.767593
1,101,2019-02-21,1.29,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,35.733466,98.739061,63.435523,98.739061
2,101,2019-02-22,1.32,96.0,1.32,53,Male,173.0,3.610061,97.22596,36.564477,36.564477,98.739061,63.435523,98.739061
3,101,2019-02-23,1.28,97.0,1.33,53,Male,173.0,3.610061,97.22596,36.841481,35.456463,99.767593,63.158519,99.767593
4,101,2019-02-24,1.33,98.0,1.36,53,Male,173.0,3.610061,97.22596,37.672492,36.841481,100.796125,62.327508,100.796125


In [ ]:
def calc_cpt_AR_HO2Sat():
    """
    Returns the CPT for the AR_HO2Sat node
    """